In [1]:
import pandas as pd
import numpy as np
import spacy
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import re
from wordcloud import WordCloud
import itertools
import collections
import nltk
import string
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import gensim
import multiprocessing
from gensim.models import Word2Vec
from multiprocessing import Process
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
import sklearn
from sklearn.cluster import KMeans

In [22]:
#data18 = pd.read_csv('carbonMarket2018_2022.csv')
data18 = pd.read_csv('carbondata_21_221031.csv')

In [23]:
data18

,User,verified,Date_Created,Follows_Count,Friends_Count,Retweet_Count,Language,Date_Tweet,Number_of_Likes,Source_of_Tweet,Tweet_Id,Tweet,Hashtags,Conversation_Id,In_reply_To,Coordinates,Place
0,CarbonCredits,False,2017-06-21 17:44:31+00:00,6799,283,0,en,2022-10-31 23:36:00+00:00,5,Twitter Web App,1587226914174935043,Nigeria is pioneering a billion-dollar worth o...,"['CarbonCredits', 'CarbonOffsets', 'Netzero', ...",1587226914174935043,NaN,NaN,NaN
1,greenbuildermag,False,2010-03-08 17:02:15+00:00,9626,5456,0,en,2022-10-31 23:35:21+00:00,0,Metricool,1587226750240571393,Green Builder Media has joined forces with hou...,"['TheSondersProject', 'netzero', 'GreenBuilder...",1587226750240571393,NaN,NaN,NaN
2,M_Costelloe,False,2012-05-03 02:19:44+00:00,604,819,0,en,2022-10-31 23:29:50+00:00,3,Twitter for iPhone,1587225362102386688,@anuearthscience @cosmicpinot Rare Earth Confe...,"['criticalminerals', 'netzero']",1587225362102386688,NaN,NaN,NaN
3,PauIRob,False,2014-02-02 11:58:19+00:00,297,97,1,en,2022-10-31 23:27:46+00:00,0,Twitter for iPad,1587224841773531137,Because your children deserve a ‘better’ world...,"['skypapers', 'cop27', 'netzero', 'ClimateEmer...",1587224841773531137,NaN,NaN,NaN
4,Bahtiar041221,False,2022-01-27 00:18:11+00:00,156,1499,0,en,2022-10-31 23:26:58+00:00,0,Twitter Web App,1587224643131256832,OMG! NZTwork is launching their HARDWARE PRESA...,"['netzerotoken', 'nztwork', 'carboncredits', '...",1587224643131256832,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355224,SusHealthcare,False,2011-07-05 09:29:14+00:00,7488,4420,4,en,2021-01-01 01:00:22+00:00,5,Hootsuite Inc.,1344810662971715586,2020 was a year for the books. Amidst the pand...,"['netzero', 'greenspaceforhealth', 'climateact...",1344810662971715586,NaN,NaN,NaN
355225,LisaKayeCAP,False,2013-08-02 04:16:44+00:00,376,832,1,en,2021-01-01 01:00:03+00:00,1,LinkedIn,1344810585658175491,The race to #NetZero is accelerating. @Ann Met...,['NetZero'],1344810585658175491,NaN,NaN,NaN
355226,thegalonthego,False,2016-04-12 23:51:26+00:00,398,1346,0,en,2021-01-01 00:32:59+00:00,1,Twitter Web App,1344803774854881280,Excellent reminder\n\n#netzero #emissions,"['netzero', 'emissions']",1344803774854881280,NaN,NaN,NaN
355227,raularteche,False,2011-11-12 12:30:32+00:00,37,91,2,es,2021-01-01 00:11:37+00:00,3,Smarp.,1344798397019152384,Anunciamos la renovación de nuestra flota con ...,"['RedLovesGreen', 'NetZero', 'CeroEmisionesNet...",1344798397019152384,NaN,NaN,NaN


### Pre-processsing

In [24]:
data18['Tweet'].iloc[3]

'Because your children deserve a ‘better’ world..\n\n#skypapers #cop27 #netzero #ClimateEmergency #ClimateScam #climatesame https://t.co/X4Q5EBtb74'

In [25]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
  
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [26]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dantr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
# the function to clean the tweet and tokenize them
def clean_tweet(tweet):
    if type(tweet) == float:
            return ""

    # turn text into lower
    test = tweet.lower()
    # remove all mentions and hashtags 
    test = re.sub("@[A-Za-z0-9_]+","", test)
    test = re.sub("#[A-Za-z0-9_]+","", test)
    #remove links
    test = re.sub(r"http\S+", "", test)
    test = re.sub(r"www.\S+", "", test)
    #remove punctuation
    test = re.sub('[()!?]', ' ', test)
    test = re.sub('\[.*?\]',' ', test)
    #remove non alphabetical characters
    test = re.sub("[^a-z0-9]"," ", test)
    #remove extra spaces
    test = re.sub(' +', ' ', test)
    
    #remove many abbreviations
    test = re.sub(r"there's", "there is", test)
    test = re.sub(r"i'm", "i am", test)
    test = re.sub(r"he's", "he is", test)
    test = re.sub(r"she's", "she is", test)
    test = re.sub(r"it's", "it is", test)
    test = re.sub(r"that's", "that is", test)
    test = re.sub(r"what's", "that is", test)
    test = re.sub(r"where's", "where is", test)
    test = re.sub(r"how's", "how is", test)
    test = re.sub(r"\'ll", " will", test)
    test = re.sub(r"\'ve", " have", test)
    test = re.sub(r"\'re", " are", test)
    test = re.sub(r"\'d", " would", test)
    test = re.sub(r"\'re", " are", test)
    test = re.sub(r"won't", "will not", test)
    test = re.sub(r"can't", "cannot", test)
    test = re.sub(r"n't", " not", test)
    test = re.sub(r"n'", "ng", test)
    test = re.sub(r"'bout", "about", test)
    test = re.sub(r"'til", "until", test)
    test = re.sub(r"\"", "", test)
    test = re.sub(r"\'", "", test)
    test = re.sub(r' s ', "", test)
    test = re.sub(r"[\[\]\\0-9()\"$#%/@;:<>{}`+=~|.!?,-]", "", test)
    test = re.sub(r"&", "", test)
    test = re.sub(r"\\n", "", test)
    
    # remove single letter words
    test = ' '.join( [w for w in test.split() if len(w)>1] )
    
    test = ' '.join( [lemma.lemmatize(x) for x in nltk.wordpunct_tokenize(test) if x not in stop_words])
    test =[lemma.lemmatize(x, nltk.corpus.reader.wordnet.VERB) for x in nltk.wordpunct_tokenize(test) if x not in stop_words]

    return test

In [28]:
clean_tweet(data18['Tweet'].iloc[4])

['omg', 'nztwork', 'launch', 'hardware', 'presale', 'soon', 'go', 'epic']

In [29]:
# clean the tweets and create two columns: tokenized tweet and whole tweet
data18["clean_tweet"]=data18["Tweet"].apply(lambda x:clean_tweet(x))
data18["cleaned_tweet"]=data18["clean_tweet"].apply(lambda x:' '.join(x))

In [30]:
data18['clean_tweet'].iloc[4]

['omg', 'nztwork', 'launch', 'hardware', 'presale', 'soon', 'go', 'epic']

In [13]:
data18['cleaned_tweet'].iloc[4]

'omg nztwork launch hardware presale soon go epic'

In [14]:
data18.shape

(355229, 19)

In [15]:
tweets = data18['clean_tweet']
tweets[:10]

0    [nigeria, pioneer, billion, dollar, worth, vol...
1    [green, builder, medium, join, force, house, s...
2                            [rare, earth, conference]
3                      [child, deserve, better, world]
4    [omg, nztwork, launch, hardware, presale, soon...
5    [omg, nztwork, launch, hardware, presale, soon...
6    [web3, startup, launch, eco, positive, nft, pr...
7    [carbon, confusion, shine, clermont, conferenc...
8    [nwt, carbon, price, approach, change, align, ...
9                                                   []
Name: clean_tweet, dtype: object

### Hashtags

In [16]:
# define a function to clean the Hashtags.
def clean_hashtags(hashtags):
    '''
    hashtags: String
              Input Data
    hashtags: String
              Output Data
           
    func: Convert hashtags to lower case
          Replace ticker symbols with space. The ticker symbols are any stock symbol that starts with $.
          Replace everything not a letter or apostrophe with space
          Removes any spaces or specified characters at the start and end of hashtags.
          
    '''
    if hashtags:
        hashtags = hashtags.lower()
        hashtags = re.sub('\$[a-zA-Z0-9]*', ' ', hashtags)
        hashtags = re.sub('[^a-zA-Z]', ' ', hashtags)
        hashtags=hashtags.strip() 
    return hashtags

In [17]:
# clean the hashtags
data18["Hashtags"]=data18["Hashtags"].astype(str)
data18["Hashtags"]=data18["Hashtags"].apply(lambda x:clean_hashtags(x))

In [18]:
data18.head()

,User,verified,Date_Created,Follows_Count,Friends_Count,Retweet_Count,Language,Date_Tweet,Number_of_Likes,Source_of_Tweet,Tweet_Id,Tweet,Hashtags,Conversation_Id,In_reply_To,Coordinates,Place,clean_tweet,cleaned_tweet
0,CarbonCredits,False,2017-06-21 17:44:31+00:00,6799,283,0,en,2022-10-31 23:36:00+00:00,5,Twitter Web App,1587226914174935043,Nigeria is pioneering a billion-dollar worth o...,carboncredits carbonoffsets netzero a...,1587226914174935043,NaN,NaN,NaN,"[nigeria, pioneer, billion, dollar, worth, vol...",nigeria pioneer billion dollar worth voluntary...
1,greenbuildermag,False,2010-03-08 17:02:15+00:00,9626,5456,0,en,2022-10-31 23:35:21+00:00,0,Metricool,1587226750240571393,Green Builder Media has joined forces with hou...,thesondersproject netzero greenbuildermedia,1587226750240571393,NaN,NaN,NaN,"[green, builder, medium, join, force, house, s...",green builder medium join force house sector p...
2,M_Costelloe,False,2012-05-03 02:19:44+00:00,604,819,0,en,2022-10-31 23:29:50+00:00,3,Twitter for iPhone,1587225362102386688,@anuearthscience @cosmicpinot Rare Earth Confe...,criticalminerals netzero,1587225362102386688,NaN,NaN,NaN,"[rare, earth, conference]",rare earth conference
3,PauIRob,False,2014-02-02 11:58:19+00:00,297,97,1,en,2022-10-31 23:27:46+00:00,0,Twitter for iPad,1587224841773531137,Because your children deserve a ‘better’ world...,skypapers cop netzero climateemerge...,1587224841773531137,NaN,NaN,NaN,"[child, deserve, better, world]",child deserve better world
4,Bahtiar041221,False,2022-01-27 00:18:11+00:00,156,1499,0,en,2022-10-31 23:26:58+00:00,0,Twitter Web App,1587224643131256832,OMG! NZTwork is launching their HARDWARE PRESA...,netzerotoken nztwork carboncredits co...,1587224643131256832,NaN,NaN,NaN,"[omg, nztwork, launch, hardware, presale, soon...",omg nztwork launch hardware presale soon go epic


### DateColumns: + month, year columns

In [19]:
data18['date'] = pd.to_datetime(data18['Date_Tweet'], format='%Y-%m-%d')
data18['month'] = data18['date'].dt.month
data18['year'] = data18['date'].dt.year

In [20]:
data18.tail()

,User,verified,Date_Created,Follows_Count,Friends_Count,Retweet_Count,Language,Date_Tweet,Number_of_Likes,Source_of_Tweet,...,Hashtags,Conversation_Id,In_reply_To,Coordinates,Place,clean_tweet,cleaned_tweet,date,month,year
355224,SusHealthcare,False,2011-07-05 09:29:14+00:00,7488,4420,4,en,2021-01-01 01:00:22+00:00,5,Hootsuite Inc.,...,netzero greenspaceforhealth climateaction,1344810662971715586,NaN,NaN,NaN,"[2020, year, book, amidst, pandemic, saw, dema...",2020 year book amidst pandemic saw demand envi...,2021-01-01 01:00:22+00:00,1,2021
355225,LisaKayeCAP,False,2013-08-02 04:16:44+00:00,376,832,1,en,2021-01-01 01:00:03+00:00,1,LinkedIn,...,netzero,1344810585658175491,NaN,NaN,NaN,"[race, accelerate, mettler, andrillon, podcast...",race accelerate mettler andrillon podcast disc...,2021-01-01 01:00:03+00:00,1,2021
355226,thegalonthego,False,2016-04-12 23:51:26+00:00,398,1346,0,en,2021-01-01 00:32:59+00:00,1,Twitter Web App,...,netzero emissions,1344803774854881280,NaN,NaN,NaN,"[excellent, reminder]",excellent reminder,2021-01-01 00:32:59+00:00,1,2021
355227,raularteche,False,2011-11-12 12:30:32+00:00,37,91,2,es,2021-01-01 00:11:37+00:00,3,Smarp.,...,redlovesgreen netzero ceroemisionesnetas...,1344798397019152384,NaN,NaN,NaN,"[anunciamos, la, renovaci, de, nuestra, flota,...",anunciamos la renovaci de nuestra flota con ve...,2021-01-01 00:11:37+00:00,1,2021
355228,raularteche,False,2011-11-12 12:30:32+00:00,37,91,2,es,2021-01-01 00:11:17+00:00,3,Smarp.,...,planeta sostenibilidad netzero ceroem...,1344798313854414849,NaN,NaN,NaN,"[en, vodafone, hemos, dado, un, paso, comprome...",en vodafone hemos dado un paso comprometimos a...,2021-01-01 00:11:17+00:00,1,2021


In [21]:
list(data18['cleaned_tweet'][(data18['year']==2022)&(data18['month']==9)][:10])

['alan ahn say still fuel future wire green speaker explain smaller efficient bedrock 24',
 'look forward attend fall festival amp connect jayman team hill charlesworth kudos jayman continue effort lead progress educate consumer',
 'safety waste storage promise tech fact change since last make mind safe reliable 24 power',
 'new climate news scale renewables mean big change electricity network',
 'emission daylight robbery double tell u 100 renewables unacceptable',
 'treat co2 pollutant basis life earth fee plant control switch global temperature ditch immediately climate crisis imaginary crisis manufacture totalitarian leave',
 'power receive highest support record week iaea conference global shift towards increasingly favourable attitude nuclear power come 50 member state plus 27 nation',
 'finance refi process use various form capital drive systematic sustainable positive change stakeholder',
 'bite gobsmacked energy expert queensland announcement break coal',
 'expose sell false a

### Turn tweets into embedding vectors 

In [ ]:
#Converting the "clean_tweet" column in the format supported by embeddings.
sent = [row for row in data18["clean_tweet"]]

#use Gensim Phrases package to automatically detect common phrases (bigrams) from a list of sentences.
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = gensim.models.phrases.Phraser(phrases)
sentences = bigram[sent]
sentences[1]

# https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial 

In [31]:
sent = [row for row in data18["clean_tweet"]]
sent

[['nigeria',
  'pioneer',
  'billion',
  'dollar',
  'worth',
  'voluntary',
  'carbon',
  'market',
  'africa',
  'africa',
  'carbon',
  'market',
  'initiative',
  'acmi',
  'estimate',
  'nigeria',
  'create',
  'million',
  'carbon',
  'credit',
  'per',
  'year'],
 ['green',
  'builder',
  'medium',
  'join',
  'force',
  'house',
  'sector',
  'pioneer',
  'thrive',
  'home',
  'builder',
  'launch',
  'home',
  'community',
  'locate',
  'fort',
  'collins',
  'colorado',
  'push',
  'limit',
  'carbon',
  'development'],
 ['rare', 'earth', 'conference'],
 ['child', 'deserve', 'better', 'world'],
 ['omg', 'nztwork', 'launch', 'hardware', 'presale', 'soon', 'go', 'epic'],
 ['omg', 'nztwork', 'launch', 'hardware', 'presale', 'soon', 'go', 'epic'],
 ['web', 'startup', 'launch', 'eco', 'positive', 'nft', 'projec'],
 ['carbon', 'confusion', 'shine', 'clermont', 'conference', 'beef', 'central'],
 ['nwt',
  'carbon',
  'price',
  'approach',
  'change',
  'align',
  'new',
  'federal'